# 模型部署————ONNX以及TensRT使用教程  
> 本代码设备基本配置如下：  
> **显卡**：TITAN Xp 12G CUDA Version: 12.8 Cuda compilation tools: 11.5（`nvcc --version`）    
> **系统**： Ubuntu 22.04.5 LTS

In [ ]:
# From: https://docs.infini-ai.com/posts/download-pytorch-from-mirror.html
# !pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu128
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 -f https://mirror.sjtu.edu.cn/pytorch-wheels/cu128
!pip install transformers==4.39.3
!pip install tensorrt
!pip install onnx onnxruntime pycuda

## ONNX推理
> [https://onnxruntime.ai/docs/get-started/](https://onnxruntime.ai/docs/get-started/)
### ONNX模型导出  
简单了解一下ONNX：**ONNX 是一种通用的模型表示格式，它通过将 PyTorch 等框架的计算图转换为 ONNX 计算图，使模型能够在不同硬件和推理引擎上通用部署**。直接使用torch中的onnx可以直接将模型进行到处，其中其具体的[模型参数](https://docs.pytorch.org/docs/stable/onnx.html)为：`torch.onnx.export(model, args=(), f=None, *, kwargs=None, verbose=None, input_names=None, output_names=None, opset_version=None, dynamo=True, external_data=True, dynamic_shapes=None, custom_translation_table=None, report=False, optimize=True, verify=False, profile=False, dump_exported_program=False, artifacts_dir='.', fallback=False, export_params=True, keep_initializers_as_inputs=False, dynamic_axes=None, training=<TrainingMode.EVAL: 0>, operator_export_type=<OperatorExportTypes.ONNX: 0>, do_constant_folding=True, custom_opsets=None, export_modules_as_functions=False, autograd_inlining=True)`

- **model (torch.nn.Module | torch.export.ExportedProgram | torch.jit.ScriptModule | torch.jit.ScriptFunction)**  
  要导出的模型。

- **args (tuple[Any, ...])**  
  示例位置参数输入。  
  - 非 Tensor 类型的参数会被硬编码进导出的模型中；  
  - Tensor 类型的参数会按照元组中出现的顺序成为模型的输入。

- **f (str | os.PathLike | None)**  
  ONNX 模型文件输出路径，例如 `"model.onnx"`。  
  此参数用于向后兼容，推荐留空（None），改用返回的 `torch.onnx.ONNXProgram` 来序列化模型。

- **kwargs (dict[str, Any] | None)**  
  示例关键字参数（可选）。

- **verbose (bool | None)**  
  是否开启详细日志。

- **input_names (Sequence[str] | None)**  
  按顺序为图的输入节点指定名称。

- **output_names (Sequence[str] | None)**  
  按顺序为图的输出节点指定名称。

- **opset_version (int | None)**  
  目标 ONNX opset 版本。应根据运行时或编译器支持的 opset 版本设置此值。  
  留空（None）表示使用推荐版本。更多信息可参考 ONNX 运算符文档。

- **dynamo (bool)**  
  是否使用 `torch.export` 的 `ExportedProgram` 导出模型，而不是 TorchScript。

- **external_data (bool)**  
  是否将模型权重单独保存为外部数据文件。  
  当模型权重大于 2GB 时必须开启此选项。  
  若为 `False`，权重会和模型结构保存在同一个 ONNX 文件中。

- **dynamic_shapes (dict[str, Any] | tuple[Any, ...] | list[Any] | None)**  
  模型输入的动态 shape 定义。详情可参考 `torch.export.export()`。  
  - 当 `dynamo=True` 时，此参数生效（并优先使用）；  
  - 注意：`dynamic_shapes` 适用于 dynamo 导出，而 `dynamic_axes` 适用于 TorchScript 导出。

- **custom_translation_table (dict[Callable, Callable | Sequence[Callable]] | None)**  
  自定义算子分解表。  
  - 字典的 key 是 fx Node 的目标算子（如 `torch.ops.aten.stft.default`），  
  - value 是使用 ONNX Script 构建该算子的函数。  
  仅在 `dynamo=True` 时有效。

- **report (bool)**  
  是否为导出过程生成 markdown 报告。仅在 `dynamo=True` 时有效。

- **optimize (bool)**  
  是否对导出的模型进行优化。仅在 `dynamo=True` 时有效。  
  默认值为 `True`。

- **verify (bool)**  
  是否使用 ONNX Runtime 验证导出的模型。仅在 `dynamo=True` 时有效。

- **profile (bool)**  
  是否对导出过程进行性能分析。仅在 `dynamo=True` 时有效。

- **dump_exported_program (bool)**  
  是否将 `torch.export.ExportedProgram` 导出到文件。  
  有助于调试导出过程。仅在 `dynamo=True` 时有效。

- **artifacts_dir (str | os.PathLike)**  
  保存调试产物（如报告和序列化的导出程序）的目录。仅在 `dynamo=True` 时有效。

- **fallback (bool)**  
  当 dynamo 导出失败时，是否回退到 TorchScript 导出方式。仅在 `dynamo=True` 时有效。  
  当开启回退时，即便提供了 `dynamic_shapes`，也推荐设置 `dynamic_axes`。

- **export_params (bool)**  
  当 `f` 被指定时：  
  - 若为 `False`，模型参数（权重）不会被导出；  
  - 也可以留空，改用返回的 `torch.onnx.ONNXProgram` 来控制序列化时对初始值的处理。

- **keep_initializers_as_inputs (bool)**  
  当 `f` 被指定时：  
  - 若为 `True`，导出图中的所有初始化器（通常是模型权重）也会被添加为输入；  
  - 若为 `False`，初始化器不会作为输入，只有用户输入会作为输入节点。  
  设置为 `True` 可在运行时提供权重；  
  设置为 `False` 可让后端进行更好的优化（如常量折叠）。

  也可以留空，改用返回的 `torch.onnx.ONNXProgram` 来控制初始值处理方式。

- **dynamic_axes (Mapping[str, Mapping[int, str]] | Mapping[str, Sequence[int]] | None)**  
  - 当 `dynamo=True` 且未启用 fallback 时，推荐使用 `dynamic_shapes` 而不是 `dynamic_axes`。  
  - 默认情况下，导出的模型会将所有输入输出 tensor 的 shape 固定为 `args` 中的 shape。  
  - 若需将 tensor 的某些轴设置为动态（运行时确定），则应传入一个 dict：

    - **KEY (str)**：输入或输出名称。必须出现在 `input_names` 或 `output_names` 中。  
    - **VALUE (dict 或 list)**：  
      - 若为 dict，key 是轴索引，value 是轴名称；  
      - 若为 list，每个元素是轴索引。

对于上述那么多参数一般而言用的比较多的也就是：1、`model`：需要处理的模型；2、`args`：输入到模型中的参数（也就是`forward`中输入的tensor）；3、`f`：存储位置/名称；4、`input_names`：输入的名称；5、`dynamic_axes`：动态输入尺寸。
### ONNX模型推理
```python
import onnxruntime as ort
ort_img_sess = ort.InferenceSession("clip_img.onnx", providers=["CPUExecutionProvider"])
ort_txt_sess = ort.InferenceSession("clip_txt.onnx", providers=["CPUExecutionProvider"])
ort_inputs_img = {"pixel_values": inputs.pixel_values.cpu().numpy().astype(np.float32)}
ort_inputs_txt = {
    "input_ids": inputs.input_ids.cpu().numpy().astype(np.int64),
    "attention_mask": inputs.attention_mask.cpu().numpy().astype(np.int64)
}

t1 = time.time()
ort_img_out = ort_img_sess.run(None, ort_inputs_img)
ort_txt_out = ort_txt_sess.run(None, ort_inputs_txt)
```
推理过程和使用torch加载权重很相似，**首先**、加载权重（`ort.InferenceSession`）；**而后**、使用加载的模型权重进行推理（`ort_img_sess.run(None, ort_inputs_img)` 后面一项需要和我到处模型的`input_names`是相对应的）
### ONNX注意事项  
1、**导出模型之后要对模型进行检查**:
```python
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)
```
2、**推理后端选择**：
```python
providers = [
    ("TensorrtExecutionProvider", {"trt_max_workspace_size": 2147483648}),
    "CUDAExecutionProvider",   # GPU推理
    "CPUExecutionProvider"     # 纯CPU推理
]
ort_session = ort.InferenceSession("model.onnx", providers=providers)
```

In [ ]:
import time
import torch
import torch.nn as nn
import onnxruntime as ort
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

import warnings
warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)
 
class ImgModelWrapper(nn.Module):
    def __init__(self, model):
        super(ImgModelWrapper, self).__init__()
        self.model = model

    def forward(self, pixel_values):
        image_features = self.model.get_image_features(pixel_values=pixel_values)
        return image_features

class TxtModelWrapper(nn.Module):
    def __init__(self, model):
        super(TxtModelWrapper, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        text_features = self.model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        return text_features

def trans_clip_onnx(clip_model_name, image_path, text= ["a photo of a cat"], cache_dir= './'):
    # 加载模型
    model = CLIPModel.from_pretrained(clip_model_name,
                                    use_safetensors=True,
                                    cache_dir= cache_dir)
    processor = CLIPProcessor.from_pretrained(clip_model_name,
                                            use_safetensors=True,
                                            cache_dir= cache_dir)
    
    # 处理输入
    image = Image.open(image_path) 
    inputs = processor(text= text, images=image, return_tensors="pt", padding='max_length')
    
    # 转换ONNX
    img_model = ImgModelWrapper(model)
    txt_model = TxtModelWrapper(model)
    
    torch.onnx.export(img_model,
                    (inputs.pixel_values),
                    "clip_img.onnx",
                    opset_version=17,
                    do_constant_folding=True,
                    input_names=['pixel_values'],
                    )
    torch.onnx.export(txt_model,
                    (inputs.input_ids, inputs.attention_mask),
                    "clip_txt.onnx",
                    opset_version=17,
                    do_constant_folding=True,
                    input_names=['input_ids', 'attention_mask'],
                    dynamic_axes={'input_ids': {0: 'batch', 1: 'seq'}, 
                                    'attention_mask': {0: 'batch', 1: 'seq'}},
                    )

def test_model_pt(clip_model_name, image_path):
    model = CLIPModel.from_pretrained(clip_model_name, use_safetensors=True)
    processor = CLIPProcessor.from_pretrained(clip_model_name, use_safetensors=True)
    model.eval()

    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")

    s_pt_time = time.time()
    with torch.no_grad():
        pt_features = model.get_image_features(pixel_values=inputs.pixel_values)
    pt_features = pt_features.cpu().numpy()
    print(f"原始推理使用时间：{time.time()- s_pt_time:.2f} 秒")

    s_onnx_time = time.time()
    ort_session = ort.InferenceSession("clip_img.onnx", providers=["CPUExecutionProvider"])
    ort_inputs = {"pixel_values": inputs.pixel_values.cpu().numpy()}
    ort_outs = ort_session.run(None, ort_inputs)
    print(f"ONNX 推理使用时间：{time.time()- s_onnx_time:.2f} 秒")

def classify_image_and_compare(clip_model_name, image_path, candidate_labels, cache_dir= './'):
    model = CLIPModel.from_pretrained(clip_model_name, use_safetensors=True,
                                      cache_dir= cache_dir)
    processor = CLIPProcessor.from_pretrained(clip_model_name, use_safetensors=True,
                                              cache_dir= cache_dir)
    model.eval()

    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=candidate_labels, images=image, return_tensors="pt", padding=True)

    # ---------------- PyTorch 推理 ----------------
    t0 = time.time()
    with torch.no_grad():
        pt_image_features = model.get_image_features(pixel_values=inputs.pixel_values)   # shape (1, D)
        pt_text_features  = model.get_text_features(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)  # shape (N, D)
    pt_time = time.time() - t0

    # 转 numpy 并做 L2 归一化
    pt_image = pt_image_features.cpu().numpy()
    pt_text  = pt_text_features.cpu().numpy()
    pt_image = pt_image / np.linalg.norm(pt_image, axis=-1, keepdims=True)
    pt_text  = pt_text  / np.linalg.norm(pt_text, axis=-1, keepdims=True)

    # 计算相似度（image @ text.T），得到每个候选标签的分数
    pt_sim = (pt_image @ pt_text.T).squeeze(0)
    pt_best_idx = int(np.argmax(pt_sim))
    pt_best_label = candidate_labels[pt_best_idx]
    pt_best_score = float(pt_sim[pt_best_idx])

    # ---------------- ONNX 推理 ----------------
    ort_img_sess = ort.InferenceSession("clip_img.onnx", providers=["CPUExecutionProvider"])
    ort_txt_sess = ort.InferenceSession("clip_txt.onnx", providers=["CPUExecutionProvider"])

    ort_inputs_img = {"pixel_values": inputs.pixel_values.cpu().numpy().astype(np.float32)}
    ort_inputs_txt = {
        "input_ids": inputs.input_ids.cpu().numpy().astype(np.int64),
        "attention_mask": inputs.attention_mask.cpu().numpy().astype(np.int64)
    }

    t1 = time.time()
    ort_img_out = ort_img_sess.run(None, ort_inputs_img)
    ort_txt_out = ort_txt_sess.run(None, ort_inputs_txt)
    onnx_time = time.time() - t1

    onnx_image = ort_img_out[0]
    onnx_text  = ort_txt_out[0] 
    onnx_image = onnx_image / np.linalg.norm(onnx_image, axis=-1, keepdims=True)
    onnx_text  = onnx_text  / np.linalg.norm(onnx_text, axis=-1, keepdims=True)

    onnx_sim = (onnx_image @ onnx_text.T).squeeze(0)
    onnx_best_idx = int(np.argmax(onnx_sim))
    onnx_best_label = candidate_labels[onnx_best_idx]
    onnx_best_score = float(onnx_sim[onnx_best_idx])

    # ---------------- 输出对比 ----------------
    print("=== PyTorch 结果 ===")
    print(f"预测标签: {pt_best_label}")
    print(f"相似度(score): {pt_best_score:.6f}")
    print(f"推理时间: {pt_time:.6f} 秒 (包含 image & text 推理)")

    print("\n=== ONNX 结果 ===")
    print(f"预测标签: {onnx_best_label}")
    print(f"相似度(score): {onnx_best_score:.6f}")
    print(f"推理时间: {onnx_time:.6f} 秒 (包含 image & text 推理)")

if __name__ == '__main__':
    clip_model = "openai/clip-vit-base-patch32"
    image_path = "./test_1.jpg"

    trans_clip_onnx(clip_model, image_path)
    candidate_labels = [
        "a photo of a cat",
        "a photo of a dog",
        "a photo of a car",
        "a photo of a building",
        "a photo of a person"
    ]
    classify_image_and_compare(clip_model, image_path, candidate_labels)

## TensoRT推理(wsl2上测试有问题)  
### Linux下安装TensoRT
在安装TensorRT前，**首先确保已经安装了CUDA和cuDNN**。**安装方式一**：`pip install tensorrt`。**安装方式二**（一定要注意显卡的nvcc版本和TensoRT市可以对应上的）：  
**首先**，下载`TensoRT`。直接去访问[网站](https://developer.nvidia.com/tensorrt#)然后直接去下载`TensorRT`（Linux直接去获取window上的下载地址然后直接到终端 `wget -c 地址`，就不需要再去上传到服务器）
> 比如说下载链接：`https://developer.download.nvidia.com/compute/machine-learning/tensorrt/10.13.3/tars/TensorRT-10.13.3.9.Linux.x86_64-gnu.cuda-12.9.tar.gz?t=eyJscyI6ImdzZW8iLCJsc2QiOiJodHRwczovL3d3dy5nb29nbGUuY29tLyJ9` 那么就可以直接：`wget -c "https://developer.download.nvidia.com/compute/machine-learning/tensorrt/10.13.3/tars/TensorRT-10.13.3.9.Linux.x86_64-gnu.cuda-12.9.tar.gz?t=eyJscyI6ImdzZW8iLCJsc2QiOiJodHRwczovL3d3dy5nb29nbGUuY29tLyJ9"      -O TensorRT-10.13.3.9.Linux.x86_64-gnu.cuda-12.9.tar.gz`

![](https://s2.loli.net/2025/10/16/dATZbpzSWYl1wyg.png)
**安装**，安装`TensoRT`  
*第一步、解压安装包*：
```bash
tar -xzvf TensorRT-10.13.3.9.Linux.x86_64-gnu.cuda-12.9.tar.gz
cd TensorRT-10.13.3.9
```
*第二步、添加环境变量*  
```bash
vim ~/.bashrc
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/hy-tmp/TensorRT-10.13.3.9/lib
export PATH=$PATH:/hy-tmp/TensorRT-10.13.3.9/bin
source ~/.bashrc
```  
*第三步、安装python包*  
```bash
python --version # Python 3.11.8
cd python
pip install tensorrt-10.13.3.9-cp311-none-linux_x86_64.whl 
```
> 在python中一般都有 `tensorrt-8.x.x.x-cp3x-none-linux_x86_64.wh` 其中 `cp`后面表示的是 python版本，tensorrt后面表示的是tensorrt版本  
> 下载为`dep`文件：  
```bash
 wget -c "https://developer.download.nvidia.com/compute/machine-learning/tensorrt/secure/8.6.0/local_repos/nv-tensorrt-local-repo-ubuntu2204-8.6.0-cuda-11.8_1.0-1_amd64.deb?__token__=exp=1760947345~hmac=e607ab2e9e2167f475d79ee9e26bcce0076c1bfdaeac93adbecaab6d79fa5dba&t=eyJscyI6IndlYnNpdGUiLCJsc2QiOiJkZXZlbG9wZXIubnZpZGlhLmNvbS90ZW5zb3JydCJ9" -O TensoRT.deb
sudo dpkg -i TensoRT.deb
sudo cp /var/nv-tensorrt-local-repo-ubuntu2204-8.6.1-cuda-11.8/nv-tensorrt-local-0628887B-keyring.gpg /usr/share/keyrings/
sudo apt-get update
sudo apt-get install tensorrt
sudo apt-get install python3-libnvinfer-dev
```


安装完毕之后可以直接通过`python -c "import tensorrt as trt; print(trt.__version__)"` `python -c "import tensorrt as trt; trt.Builder(trt.Logger(trt.Logger.WARNING))"`去验证是否安装成功。
### TensoRT推理
TensorRT有自己的一套推理流程，我们在使用PyTorch或TensorFlow导出模型权重后，需要进一步转换。TensorRT最终需要的是一个TensorRT Engine，这个Engine是由TensorRT的Builder构建，而Builder需要一个TensorRT Network。TensorRT Network是由TensorRT Parser解析的ONNX模型构建的。
![](https://s2.loli.net/2025/10/17/VGwpv8hWS3XIfem.png)
**第一步、转换为TensoRT Engine**。需要完成操作：1、转换并填充Network对象；2、编写构建配置对象（Config）；3、编写优化配置对象（Optimization Profile）整个过程参考下面的`build_engine_from_onnx`函数，对于其中更加具体的细节描述如下：
> 对于该函数主要3个核心内容：1、创建我的3部分核心内容：`builder`、`config`、`parser`。构建、配置、解析。

**1、使用显式批处理**：显式批处理（explicit batch）和隐式批处理（implicit batch）。假设网络的输入张量形状为`(n, c, h, w)`，在隐式批处理模式下，网络只需要指定输入形状为`(c, h, w)`，批次维度是隐式的，并且在运行时可动态指定；在显式批处理模式下，批量维度需要网络显示定义，甚至可以不用批量维度，对于动态批量大小的需求，可以使用Optimization Profile配置动态形状
>`network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))`  
  
**2、动态形状**：直接通过`set_shape`去设定尺寸。一般而言其参数为：输入节点名称，可接受的最小输入尺寸，最优的输入尺寸，可接受的最大输入尺寸


**第二步、使用TensoRT进行推理**。